<big><b>List of satellites that are currently above algerian territory<b/><big/>

This notebook searches among satellites that are in space using their TLE to retrieve and display the ones that are located above the algerian area

<big> <b>Installing required packages <b/><big/>

In [121]:
#install required packages
import sys 

!{sys.executable} -m pip install skyfield --user
!{sys.executable} -m pip install reverse_geocoder --user

# Non necessary packages
#!{sys.executable} -m pip install pyorbital --user 
#!{sys.executable} -m pip install pprint --user
#!{sys.executable} -m pip install pyephem --user 
#!{sys.executable} -m pip install geopy --user 
#!{sys.executable} -m pip install arcgis --user 
#!{sys.executable} -m pip install poliastro --user

     |████████████████████████████████| 245kB 186kB/s eta 0:00:01
  Created wheel for skyfield: filename=skyfield-1.16-cp37-none-any.whl size=273610 sha256=de90652e912bec2c386ce43d35981ea03fb11ace12e0dcee0b3e277a50db80b7
  Stored in directory: /home/tarek/.cache/pip/wheels/4b/27/4c/29aa7d60909618f39aca67c7170969e2ec6a8c7aebe0c1a455
  Created wheel for sgp4: filename=sgp4-1.4-cp37-none-any.whl size=35409 sha256=9d7eac2354727883e32f5aca954fe5f4a35e62b45ea94e8a47c1117181f6dc85
  Stored in directory: /home/tarek/.cache/pip/wheels/6d/e2/42/5dc20daf2ba62ae03dc8abe10744ee67d9452df447dac561db
Successfully built skyfield sgp4


<big><b>Importing needed packages<b/><big/>

In [8]:
# Import required packaged

import os
import re
import urllib
from skyfield import api as sf
from skyfield.api import EarthSatellite, Topos, load
import reverse_geocoder as rg
from datetime import datetime
#from pyorbital.orbital import Orbital
#from pyorbital import tlefile

<big> <b>Retrieving TLE from Celestrak using Skyfield package<b/><big/>

In [9]:
#stations_url = 'http://celestrak.com/NORAD/elements/stations.txt'
#satellites = load.tle(stations_url)

datadir = os.environ['HOME']+'/programming/python/astro/skyfield-data/'
loader = sf.Loader(datadir, expire=False)

# Satellite TLEs.
celestrak = 'http://celestrak.com/NORAD/elements/'
geosats = loader.tle(celestrak+'geo.txt')
gpssats = loader.tle(celestrak+'supplemental/gps.txt')
sciencesats = loader.tle(celestrak+'science.txt')
stationsats = loader.tle(celestrak+'stations.txt')
tdrss = loader.tle(celestrak+'tdrss.txt')
visualsats = loader.tle(celestrak+'visual.txt')

# Make catalogs indexable by either name or catalog number.
# Also create sats, a merge of the individual ones.
satcats = [geosats, gpssats, sciencesats, stationsats, tdrss, visualsats]
sats = {}
catalog = {}
for cat in satcats:
    names = [key for key in cat.keys()]
    for name in names:
#        print(name)
        sat = cat[name]
        satnum = sat.model.satnum
        cat[satnum]  = sat
        sats[satnum] = sat
        sats[name]   = sat

<big> <b>Function to retrieve Satellites by both name and ID<b/> <big/>

In [10]:
def getsat(satid):
    """
    Return a skyfield EarthSatellite.
    <satid> is case independent if it is a satellite name (str).
    Retrieve directly from Celestrak by ID if not in local database.
    """
    if isinstance(satid, str):
        satid = satid.upper()
    if satid in sats.keys():
        return sats[satid]
    if not isinstance(satid, int):
        msg = 'satid must be an integer for satellites not in the local set'
        raise Exception(msg)
    base = 'http://celestrak.com/cgi-bin/TLE.pl?CATNR='
    url = base + str(satid)
    with urllib.request.urlopen(url) as fd:
        lines = fd.readlines()
    for k, line in enumerate(lines):
        if 'PRE' in line.decode():
            name = lines[k+1].decode().strip()
            if name == 'No TLE found':
                msg = '%i is not in the CelesTrak database!' % satid
                raise Exception(msg)
            tle1 = lines[k+2].decode().strip()
            tle2 = lines[k+3].decode().strip()
            break
    sat = sf.EarthSatellite(tle1, tle2, name)
    return sat

<big> <b> Function to get Satellite Latitude and Longitude <b/><big/>

In [11]:
def getlonlat(subpoint):
    """
    Function used to retrieve the Satellite's longitude and latitude 
    from the EarthSattelite.subpoint object
    And convert them to degree decimal
    """
    
    #LONGITUDE EarthSattelite.Angle object -> Degree Decimal
    lon = subpoint.longitude
    strlongitude= str(lon)
    lonstr =  re.split('[deg\'"]', strlongitude)
    londeg, lonminute, lonseconde = lonstr[0], lonstr[3], lonstr[4]
    longitude = (float(londeg) + float(lonminute)/60 + float(lonseconde)/(60*60))

    #LATITUDE EarthSattelite.Angle object -> Degree Decimal
    lat = subpoint.latitude
    strlatitude= str(lat)
    latstr =  re.split('[deg\'"]', strlatitude)
    latdeg, latminute, latseconde = latstr[0], latstr[3], latstr[4]
    latitude = (float(latdeg) + float(latminute)/60 + float(latseconde)/(60*60))
    
    return longitude, latitude

<big><b> Retrieving data of Satellites that are above Algeria <b/><big/>

In [25]:
SatList = []
for name in names:
    sat = getsat(name)
    ts = load.timescale()
    t = ts.now()
    geometry = sat.at(t)
    subpoint = geometry.subpoint()
    alt = float(subpoint.elevation.km)
    if alt < 1000:
        longitude, latitude = getlonlat(subpoint)
#        print('Longitude :', longitude, 'Latitude', latitude)
        coordinates = (longitude, latitude) # Defining the satellite's coordiantes
        SatLocation = rg.search(coordinates) # Reverse geocoding to get the satellite's projection
        SatLocation[0]
        if SatLocation[0]['cc'] == 'DZ':
            altitude = int(subpoint.elevation.km)
            SatList.append({'Satellite':name,'Elevation': altitude})
            print('Satellite (Name/ID): ', name, '-', 'Hauteur (km): ', altitude)            

Satellite (Name/ID):  14032 - Hauteur (km):  518
Satellite (Name/ID):  COSMOS 1455 - Hauteur (km):  518
Satellite (Name/ID):  24298 - Hauteur (km):  846


<big> <b> Displaying retrieved data <b><big/>

In [27]:
for elem in SatList:
    print('Satellite (Name/ID): ', elem['Satellite'], ' - ', 'Elevation (km): ', elem['Elevation'])

Satellite (Name/ID):  14032  -  Elevation (km):  518
Satellite (Name/ID):  COSMOS 1455  -  Elevation (km):  518
Satellite (Name/ID):  24298  -  Elevation (km):  846
